# Light model V2 (train 14 + 18 val 17)

In [1]:
import numpy as np
import tensorflow as tf
import random as python_random
import os
import datetime

seed_value=1

os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
python_random.seed(seed_value)
tf.random.set_seed(seed_value)

import sys
sys.path.append('D:/Paper_1/Importance/Olden_for_Keras')
from Olden_Keras import Olden
sys.path.append('d:/Paper_1/Syntax/afunc')
from plotaccdl import plot_history, plot_acc, plot_acc_no_colorbar, error_plot, train_preparation
from MLP_ARC import *
from MLP_acc import acc_report

import keras
from keras import regularizers
from keras import backend as K
from keras.models import load_model, Model
from keras.layers import Dropout, BatchNormalization, Activation, Input, Dense

from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy.stats as stats
import glob


from matplotlib import pyplot as plt
import pandas as pd

## 1. Data Generation

In [2]:
df=pd.read_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dt0_snow_v2.csv')

# Date Correction
year_corr=df.iloc[(df.loc[df['dayofyear']==1].index+1),]['year']
df.iloc[(df.loc[df['dayofyear']==1].index),0]=list(year_corr)

year_01=df.iloc[(df.loc[df['dayofyear']==1].index),]['year']
year_02=df.iloc[(df.loc[df['dayofyear']==1].index+1),]['year']

# Calculating dates
df_dates=[]

for df_row_i in range(df.shape[0]):
    year = df.iloc[df_row_i,]['year']
    days = df.iloc[df_row_i,]['dayofyear']
    date_temp = datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(days) - 1)
    
    df_dates.append(date_temp)
    
df['date'] = pd.Series(df_dates, index=df.index)

# Calculating lags
#Find switchs
switch_flag=np.asarray(list(df['year'])[1:])-np.asarray(list(df['year'])[0:-1])
switch=np.where(switch_flag<(-1)) # use switch[0] to have the array

begin=np.append(0,(switch[0]+1))
begin=np.append(begin, df.shape[0])

melt_l1=np.array([])

for i in range(len(begin)-1):
    melt_lag_1=np.append(-99,df.iloc[begin[i]:(begin[i+1]-1),]['melt_0'])
    melt_l1=np.append(melt_l1,melt_lag_1)
    
df['melt_l1'] = pd.Series(melt_l1, index=df.index)

# Wash
df_wash=df.loc[(df['albedo_0']!=-99)&(df['melt_0']!=-99)&(df['melt_l1']!=-99)]

# Temporal melt change
df_wash['melt_TC']=df_wash['melt_0']-df_wash['melt_l1']
# Albedo <= 0.95
df_wash['Albedo_A']=df_wash["albedo_0"]-df_wash["delta_albedo"]

Unreal=np.where(df_wash['Albedo_A']>0.95)[0]
#Unreal_Alb=list(np.append(Unreal,(Unreal+1)))

# Correcting delta_albedo
df_wash['delta_albedo']=df_wash['delta_albedo']*(-1)

D:\Software\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Software\anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Software\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [3]:
df_wash2=df_wash.drop(df_wash.index[list(set(Unreal))],axis=0,inplace=False)
df_wash2.describe()

print(df.shape)
print(df_wash.shape)
print(df_wash2.shape)
print(Unreal.shape)
#print(len(Unreal_Alb))
#print(len(set(Unreal_Alb)))
#print(df_wash.shape[0]-len(Unreal_Alb))
#print(df_wash.shape[0]-len(set(Unreal_Alb)))

(197750, 14)
(197680, 16)
(180782, 16)
(16898,)


In [4]:
# Train/Test separation
train=df_wash2.loc[(df_wash2['AWS_ID']==1)|(df_wash2['AWS_ID']==2)|(df_wash2['AWS_ID']==4)]
test=df_wash.loc[(df_wash['AWS_ID']==3)]

print(train.shape)
print(test.shape)

train.to_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_train_NM_14_17.csv',index_label=False)
test.to_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_test_NM_18.csv',index_label=False)

(167639, 16)
(15162, 16)


In [5]:
# Train/Test separation
train=df_wash2.loc[(df_wash2['AWS_ID']==1)|(df_wash2['AWS_ID']==2)]
test=df_wash.loc[(df_wash['AWS_ID']==3)]

print(train.shape)
print(test.shape)

train.to_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_train_no_NM_14_17.csv',index_label=False)
test.to_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_test_no_NM_18.csv',index_label=False)

(55381, 16)
(15162, 16)


In [6]:
# Train/Test separation
train=df_wash2.loc[(df_wash2['AWS_ID']==1)|(df_wash2['AWS_ID']==3)]
test=df_wash.loc[(df_wash['AWS_ID']==2)]

print(train.shape)
print(test.shape)

train.to_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_train_no_NM_14_18.csv',index_label=False)
test.to_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_test_no_NM_17.csv',index_label=False)

(50239, 16)
(19992, 16)


In [7]:
train_ori1=pd.read_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dt0_snow_v2_train_alb_cor.csv')
test1=pd.read_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dt0_snow_v2_test_full.csv')

train_ori2=pd.read_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_train_NM_14_17.csv')
test2=pd.read_csv('D:/Paper_1/Data/allaws_dy_nnmatrix_dT0_snow_v2_test_NM_18.csv')

print(test1.shape)
print(test2.shape)
print(train_ori1.shape)
print(train_ori2.shape)

(15162, 16)
(15162, 16)
(167639, 16)
(167639, 16)


In [8]:
# pd.concat([train_ori1,train_ori2]).drop_duplicates(keep=False)
# pd.concat([test1,test2]).drop_duplicates(keep=False)

In [9]:
x=train_ori1.drop('delta_albedo',1)
y=train_ori2.drop('delta_albedo',1)

pd.concat([x,y]).drop_duplicates(keep=False)

,year,dayofyear,albedo_0,temp2m,shortwave_in,longwave_in,melt_0,delta_melt,delta_temp2m,AWS_ID,meltflag,date,melt_l1,melt_TC,Albedo_A


In [14]:
x=test1.drop('delta_albedo',1)
y=test2.drop('delta_albedo',1)

pd.concat([x,y]).drop_duplicates(keep=False)

,year,dayofyear,albedo_0,temp2m,shortwave_in,longwave_in,melt_0,delta_melt,delta_temp2m,AWS_ID,meltflag,date,melt_l1,melt_TC,Albedo_A


In [15]:
x=train_ori1.drop('delta_albedo',1)
y=train_ori2.drop('delta_albedo',1)
for i in range(x.shape[0]):
    
    if train_ori1.iloc[i, df.columns.get_loc('delta_albedo')] != -(train_ori2.iloc[i, df.columns.get_loc('delta_albedo')]):
        #print(i)
        print(train_ori1.iloc[i, df.columns.get_loc('delta_albedo')])
        print(train_ori2.iloc[i, df.columns.get_loc('delta_albedo')])
        print('='*20)

x=test1.drop('delta_albedo',1)
y=test2.drop('delta_albedo',1)
for i in range(y.shape[0]):
    
    if test1.iloc[i, df.columns.get_loc('delta_albedo')] != -(test2.iloc[i, df.columns.get_loc('delta_albedo')]):
        #print(i)
        print(test1.iloc[i, df.columns.get_loc('delta_albedo')])
        print(test2.iloc[i, df.columns.get_loc('delta_albedo')])
        print('='*20)

In [16]:
x=train_ori1.drop('delta_albedo',1)
y=train_ori2.drop('delta_albedo',1)
for i in range(x.shape[0]):
    
    if train_ori1.iloc[i, df.columns.get_loc('shortwave_in')] != (train_ori2.iloc[i, df.columns.get_loc('shortwave_in')]):
        #print(i)
        print(train_ori1.iloc[i, df.columns.get_loc('shortwave_in')])
        print(train_ori2.iloc[i, df.columns.get_loc('shortwave_in')])
        print('='*20)
 
x=test1.drop('delta_albedo',1)
y=test2.drop('delta_albedo',1)
for i in range(y.shape[0]):
    
    if test1.iloc[i, df.columns.get_loc('shortwave_in')] != (test2.iloc[i, df.columns.get_loc('shortwave_in')]):
        #print(i)
        print(test1.iloc[i, df.columns.get_loc('shortwave_in')])
        print(test2.iloc[i, df.columns.get_loc('shortwave_in')])
        print('='*20)

In [17]:
x=train_ori1.drop('delta_albedo',1)
y=train_ori2.drop('delta_albedo',1)
for i in range(x.shape[0]):
    
    if train_ori1.iloc[i, df.columns.get_loc('melt_0')] != (train_ori2.iloc[i, df.columns.get_loc('melt_0')]):
        #print(i)
        print(train_ori1.iloc[i, df.columns.get_loc('melt_0')])
        print(train_ori2.iloc[i, df.columns.get_loc('melt_0')])
        print('='*20)

x=test1.drop('delta_albedo',1)
y=test2.drop('delta_albedo',1)
for i in range(y.shape[0]):
    
    if test1.iloc[i, df.columns.get_loc('melt_0')] != (test2.iloc[i, df.columns.get_loc('melt_0')]):
        #print(i)
        print(test1.iloc[i, df.columns.get_loc('melt_0')])
        print(test2.iloc[i, df.columns.get_loc('melt_0')])
        print('='*20)

# ================ End Here ============================